In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import numpy as np

from category_encoders.target_encoder import TargetEncoder
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb

# Анализ данных

In [2]:
import analytics as als

In [3]:
train = pd.read_csv('application_train.csv')
test = pd.read_csv('application_test.csv')

In [4]:
train.shape

(307511, 122)

In [5]:
test.shape

(48744, 121)

In [6]:
train

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
test

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,456221,Cash loans,F,N,Y,0,121500.0,412560.0,17473.5,270000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
48740,456222,Cash loans,F,N,N,2,157500.0,622413.0,31909.5,495000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
48741,456223,Cash loans,F,Y,Y,1,202500.0,315000.0,33205.5,315000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,3.0,1.0
48742,456224,Cash loans,M,N,N,0,225000.0,450000.0,25128.0,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0


In [8]:
als.check_missing_values(train, 15, 1)

133770    61
244833    61
150206    61
69707     61
269786    61
269492    61
116937    61
185713    61
197736    61
87399     60
7077      60
115346    60
201306    60
146561    60
68013     60
dtype: int64

In [9]:
als.check_missing_values(train, 15, 0)

COMMONAREA_MEDI             214865
COMMONAREA_AVG              214865
COMMONAREA_MODE             214865
NONLIVINGAPARTMENTS_MODE    213514
NONLIVINGAPARTMENTS_MEDI    213514
NONLIVINGAPARTMENTS_AVG     213514
FONDKAPREMONT_MODE          210295
LIVINGAPARTMENTS_MEDI       210199
LIVINGAPARTMENTS_MODE       210199
LIVINGAPARTMENTS_AVG        210199
FLOORSMIN_MEDI              208642
FLOORSMIN_MODE              208642
FLOORSMIN_AVG               208642
YEARS_BUILD_MEDI            204488
YEARS_BUILD_AVG             204488
dtype: int64

In [10]:
als.check_missing_values(test, 15, 1)

45915    61
46262    58
3220     58
9149     58
11311    58
8612     58
43634    58
23933    58
26280    57
21281    57
2530     57
12601    57
19184    57
45900    57
29293    57
dtype: int64

In [11]:
als.check_missing_values(test, 15, 0)

COMMONAREA_MEDI             33495
COMMONAREA_AVG              33495
COMMONAREA_MODE             33495
NONLIVINGAPARTMENTS_MODE    33347
NONLIVINGAPARTMENTS_MEDI    33347
NONLIVINGAPARTMENTS_AVG     33347
FONDKAPREMONT_MODE          32797
LIVINGAPARTMENTS_AVG        32780
LIVINGAPARTMENTS_MEDI       32780
LIVINGAPARTMENTS_MODE       32780
FLOORSMIN_MEDI              32466
FLOORSMIN_MODE              32466
FLOORSMIN_AVG               32466
OWN_CAR_AGE                 32312
YEARS_BUILD_MEDI            31818
dtype: int64

##### В данных много пропусков, поэтому стоит удалить те объекты и признаки, в которых много пропусков.

In [4]:
train_omissions = als.get_features_with_omissions(train)
test_omissions = als.get_features_with_omissions(test)

In [14]:
als.get_unique_value(train, train_omissions)

AMT_ANNUITY <class 'numpy.float64'> [24700.5 35698.5  6750.  ... 71986.5 58770.  77809.5]
AMT_GOODS_PRICE <class 'numpy.float64'> [ 351000.  1129500.   135000.  ...  453465.   143977.5  743863.5]
NAME_TYPE_SUITE <class 'str'> ['Unaccompanied' 'Family' 'Spouse, partner' 'Children' 'Other_A' nan
 'Other_B' 'Group of people']
OWN_CAR_AGE <class 'numpy.float64'> [nan 26. 17.  8. 23.  7. 14.  1.  3. 13. 10.  2. 21. 20. 22. 65.  5. 11.
  9.  4. 15. 12. 25.  6. 64.  0. 27. 19. 28. 18. 16. 30. 36. 24. 39. 29.
 40. 41. 31. 42. 37. 35. 34. 33. 54. 38. 32. 43. 55. 49. 44. 63. 46. 45.
 48. 56. 51. 57. 69. 50. 47. 91. 52.]
OCCUPATION_TYPE <class 'str'> ['Laborers' 'Core staff' 'Accountants' 'Managers' nan 'Drivers'
 'Sales staff' 'Cleaning staff' 'Cooking staff' 'Private service staff'
 'Medicine staff' 'Security staff' 'High skill tech staff'
 'Waiters/barmen staff' 'Low-skill Laborers' 'Realty agents' 'Secretaries'
 'IT staff' 'HR staff']
CNT_FAM_MEMBERS <class 'numpy.float64'> [ 1.  2.  3.  4.  

BASEMENTAREA_MEDI <class 'numpy.float64'> [0.0369 0.0529    nan ... 0.3482 0.274  0.2733]
YEARS_BEGINEXPLUATATION_MEDI <class 'numpy.float64'> [0.9722 0.9851    nan 0.9811 0.9806 0.9985 0.9881 0.9781 0.9786 0.9876
 0.9861 0.9732 0.9831 0.9697 0.9901 0.9801 0.9762 0.996  0.998  0.9846
 0.9886 0.9136 0.9841 0.9856 0.9955 0.9677 0.9891 0.9752 0.9826 0.9727
 0.9578 0.9742 0.9707 0.9975 0.999  0.9816 0.9866 0.9836 0.994  0.9757
 0.9737 0.9791 0.9821 0.9682 0.9945 0.9771 0.9747 0.9776 0.9796 0.9896
 0.9692 0.9518 0.995  0.9995 0.9414 0.993  0.9702 0.9921 0.9916 0.9712
 0.4933 0.9871 0.9672 0.9911 0.9608 0.9906 0.9503 0.9598 0.997  0.9583
 0.9767 0.929  0.9935 0.9617 0.9563 0.9965 0.9925 0.9508 0.9717 0.9687
 0.9667 0.9359 0.9573 0.9588 0.9513 0.9642 0.9553 0.9603 0.9622 0.9637
 0.     0.9568 0.9652 0.9523 0.4953 0.9473 0.9071 0.9593 0.9627 0.9613
 0.9439 0.9434 0.9488 0.4973 0.9459 0.9334 0.9657 0.9632 0.9424 0.9548
 0.9449 0.4913 0.9647 0.9662 0.9468 0.9001 0.9324 0.9389 1.     0.9493
 0.94

In [17]:
als.get_unique_value(test, test_omissions)

AMT_ANNUITY <class 'numpy.float64'> [20560.5 17370.  69777.  ... 37917.  40824.  46660.5]
NAME_TYPE_SUITE <class 'str'> ['Unaccompanied' nan 'Family' 'Spouse, partner' 'Group of people'
 'Other_B' 'Children' 'Other_A']
OWN_CAR_AGE <class 'numpy.float64'> [nan  5. 16. 10.  3. 14. 11. 15.  8. 20. 22.  1. 19.  0. 21. 13.  4.  6.
  9. 42. 24.  2. 23. 17. 29. 35. 31. 65.  7. 26. 25. 18. 12. 30. 36. 34.
 32. 28. 39. 27. 37. 41. 40. 50. 38. 56. 33. 74. 43. 44. 55. 49. 52.]
OCCUPATION_TYPE <class 'float'> [nan 'Low-skill Laborers' 'Drivers' 'Sales staff' 'High skill tech staff'
 'Core staff' 'Laborers' 'Managers' 'Accountants' 'Medicine staff'
 'Security staff' 'Private service staff' 'Secretaries' 'Cleaning staff'
 'Cooking staff' 'HR staff' 'Waiters/barmen staff' 'Realty agents'
 'IT staff']
EXT_SOURCE_1 <class 'numpy.float64'> [0.75261449 0.5649902         nan ... 0.17467087 0.7335035  0.37309008]
EXT_SOURCE_2 <class 'numpy.float64'> [0.78965435 0.29165553 0.69978683 ... 0.63276955 0.445700

In [11]:
len(als.get_features_with_omissions(train))

67

# Обработка пропусков

In [3]:
import treatment as pso

##### Удаляем объекты, где много пропусков (больше 30 признаков пропущенно)

In [6]:
to_drop = pso.get_obj_to_drop(train, 30, 1)

In [7]:
len(to_drop)

160717

In [8]:
train.shape

(307511, 122)

In [9]:
train.drop(train.index[to_drop], inplace=True)

In [251]:
train.shape

(146794, 122)

##### Удаляем признаки, которые пропущены в большом количестве наблюдение (более, чем в 40000 объектах)

In [10]:
to_drop = pso.get_obj_to_drop(train, 20000, 0)

In [11]:
len(to_drop)

23

In [437]:
to_drop

['OWN_CAR_AGE',
 'EXT_SOURCE_1',
 'COMMONAREA_AVG',
 'COMMONAREA_MEDI',
 'COMMONAREA_MODE',
 'NONLIVINGAPARTMENTS_MEDI',
 'NONLIVINGAPARTMENTS_MODE',
 'NONLIVINGAPARTMENTS_AVG',
 'FONDKAPREMONT_MODE',
 'LIVINGAPARTMENTS_MEDI',
 'LIVINGAPARTMENTS_AVG',
 'LIVINGAPARTMENTS_MODE',
 'FLOORSMIN_MEDI',
 'FLOORSMIN_MODE',
 'FLOORSMIN_AVG',
 'OCCUPATION_TYPE',
 'YEARS_BUILD_AVG',
 'YEARS_BUILD_MEDI',
 'YEARS_BUILD_MODE',
 'EXT_SOURCE_3',
 'LANDAREA_MODE',
 'LANDAREA_MEDI',
 'LANDAREA_AVG']

##### Несмотря на большое количество пропусков, не стоит удалять фичи 'EXT_SOURCE_3',  'EXT_SOURCE_1', 'OCCUPATION_TYPE', 'OWN_CAR_AGE' 

In [12]:
to_drop = list(set(to_drop) - set(['EXT_SOURCE_3',  'EXT_SOURCE_1', 'OCCUPATION_TYPE', 'OWN_CAR_AGE']))

In [13]:
train.drop(to_drop, axis='columns', inplace=True)

In [440]:
train.shape

(146794, 103)

In [283]:
als.check_missing_values(train, 15, 0)

OWN_CAR_AGE                   98529
EXT_SOURCE_1                  79136
OCCUPATION_TYPE               45688
EXT_SOURCE_3                  26229
BASEMENTAREA_AVG              19892
BASEMENTAREA_MODE             19892
BASEMENTAREA_MEDI             19892
AMT_REQ_CREDIT_BUREAU_YEAR    16647
AMT_REQ_CREDIT_BUREAU_MON     16647
AMT_REQ_CREDIT_BUREAU_WEEK    16647
AMT_REQ_CREDIT_BUREAU_DAY     16647
AMT_REQ_CREDIT_BUREAU_HOUR    16647
AMT_REQ_CREDIT_BUREAU_QRT     16647
NONLIVINGAREA_MEDI            10486
NONLIVINGAREA_MODE            10486
dtype: int64

In [284]:
als.check_missing_values(train, 15, 1)

245902    23
71810     22
243985    21
112232    20
189891    20
75931     20
219934    20
141154    20
298320    20
173405    20
100922    19
221405    19
179090    19
220789    19
142191    19
dtype: int64

##### Общее количество пропусков не такое уж большое. Можно остальные пропуски заполним простым методом. Числовые признаки заполним числом 9999, а строковые новой категорией.

In [14]:
train.index = np.arange(len(train))

In [15]:
feature_str, feature_int = als.get_lists_type_feature(train, list(train))

In [16]:
pso.fill_gaps(train, feature_str, 'new_category')
pso.fill_gaps(train, feature_int, 9999)

##### Можно попробовать несколько кодировок для категориальных признаков. Попробуем One-hot и TargetEncoding

In [288]:
train_oh = pd.get_dummies(train, columns=feature_str)

In [289]:
# Обработка названий признаков для бустинга
train_oh.columns = ["".join (i if i.isalnum() else "_" for i in str(x)) for x in train_oh.columns]

In [290]:
X = train_oh.drop(columns=['SK_ID_CURR', 'TARGET'])
y = train_oh['TARGET']

In [291]:
feature_str, feature_int = als.get_lists_type_feature(train_oh, list(train_oh))

In [292]:
# Обработка типов для бустинга

In [293]:
for feature in feature_str:
    X[feature] = X[feature].astype('float64')

In [294]:
kf = StratifiedKFold(n_splits=5,  shuffle=True)

In [316]:
for train_indices, test_indices in kf.split(X, y):
    X_train, X_val = X.iloc[train_indices], X.iloc[test_indices]
    y_train, y_val = y.iloc[train_indices], y.iloc[test_indices]
    lgbmc = lgb.LGBMClassifier(mobjective='binary',
                               max_depth=30, learning_rate=0.05,
                               n_estimators=100, class_weight='balanced',
                               subsample=0.8,
                               colsample_bytree=0.8,
                               random_state=42,
                               importance_type='gain')
    model = lgbmc.fit(X_train, y_train)
    y_pred_train = model.predict(X_train)
    score_train = roc_auc_score(y_train, y_pred_train)
    print(f'train_score {score_train}')   
    y_pred_val = model.predict(X_val)
    score_val = roc_auc_score(y_val, y_pred_val)
    print(f'test_score {score_val}')

train_score 0.7238682787649355
test_score 0.6910601473566467
train_score 0.7247322177417187
test_score 0.6904812622587597
train_score 0.7264696531769739
test_score 0.683238666664937
train_score 0.7240603967050608
test_score 0.6809894361221029
train_score 0.728262895275309
test_score 0.6815973764435114


#### Попробуем другое кодирование

In [17]:
train_te = train

In [18]:
X = train_te.drop(columns=['SK_ID_CURR', 'TARGET'])
y = train_te['TARGET']

In [19]:
feature_str, feature_int = als.get_lists_type_feature(train_te, list(train_te))

In [20]:
TE_encoder = TargetEncoder(cols=feature_str)

In [298]:
for train_indices, test_indices in kf.split(X, y):
    X.iloc[train_indices] = TE_encoder.fit_transform(X.iloc[train_indices], y.iloc[train_indices])
    X.iloc[test_indices] = TE_encoder.transform(X.iloc[test_indices])

In [299]:
for feature in feature_str:
    X[feature] = X[feature].astype('float64')

In [300]:
for train_indices, test_indices in kf.split(X, y):
    X_train, X_val = X.iloc[train_indices], X.iloc[test_indices]
    y_train, y_val = y.iloc[train_indices], y.iloc[test_indices]
    lgbmc = lgb.LGBMClassifier(mobjective='binary',
                               max_depth=30, learning_rate=0.05,
                               n_estimators=100, class_weight='balanced',
                               subsample=0.8,
                               colsample_bytree=0.8,
                               random_state=42,
                               importance_type='gain')
    model = lgbmc.fit(X_train, y_train)
    y_pred_train = model.predict(X_train)
    score_train = roc_auc_score(y_train, y_pred_train)
    print(f'train_score {score_train}')   
    y_pred_val = model.predict(X_val)
    score_val = roc_auc_score(y_val, y_pred_val)
    print(f'test_score {score_val}')

train_score 0.723970040126868
test_score 0.6869530365975267
train_score 0.7225587454992717
test_score 0.6870212795781315
train_score 0.7250530919311193
test_score 0.6817017814190354
train_score 0.7255055306923485
test_score 0.6890878623919889
train_score 0.7236552779450355
test_score 0.6881505895178874


##### Обучаем тест с One-hot.

In [26]:
test.drop(to_drop, axis='columns', inplace=True)

In [27]:
feature_sts, feature_int = als.get_lists_type_feature(test, list(test))

In [28]:
pso.fill_gaps(test, feature_str, 'new_category')
pso.fill_gaps(test, feature_int, 9999)

In [387]:
data_oh = pd.concat([train, test])

In [388]:
test.shape

(195538, 103)

In [389]:
train.shape

(146794, 103)

In [395]:
data_oh.index = np.arange(len(data_oh))

In [396]:
feature_str, feature_int = als.get_lists_type_feature(data_oh, list(data_oh))

In [397]:
data_oh = pd.get_dummies(data_oh, columns=feature_str)

In [400]:
data_oh.columns = ["".join (i if i.isalnum() else "_" for i in str(x)) for x in data_oh.columns]

In [402]:
feature_str, feature_int = als.get_lists_type_feature(data_oh, list(data_oh))

In [403]:
for feature in feature_str:
    data_oh[feature] = data_oh[feature].astype('float64')

In [421]:
train_oh = data_oh.iloc[:146794, :]
test_oh = data_oh.iloc[146794:, :]

In [422]:
X = data_oh.drop(columns=['TARGET'])
y = data_oh['TARGET']
test_oh = test_oh.drop(columns=['TARGET'])

In [423]:
kf = StratifiedKFold(n_splits=5,  shuffle=True)

In [424]:
for train_indices, test_indices in kf.split(X, y):
    X_train, X_val = X.iloc[train_indices], X.iloc[test_indices]
    y_train, y_val = y.iloc[train_indices], y.iloc[test_indices]
    lgbmc = lgb.LGBMClassifier(mobjective='binary',
                               max_depth=30, learning_rate=0.05,
                               n_estimators=100, class_weight='balanced',
                               subsample=0.8,
                               colsample_bytree=0.8,
                               random_state=42,
                               importance_type='gain')
    model = lgbmc.fit(X_train, y_train)
    y_pred_train = model.predict(X_train)
    score_train = roc_auc_score(y_train, y_pred_train)
    print(f'train_score {score_train}')   
    y_pred_val = model.predict(X_val)
    score_val = roc_auc_score(y_val, y_pred_val)
    print(f'test_score {score_val}')

train_score 0.7238881304922098
test_score 0.6915656945937623
train_score 0.7256108060420685
test_score 0.6759129226641621
train_score 0.7250589139248536
test_score 0.6894400063794557
train_score 0.7264687048379088
test_score 0.6840253522975047
train_score 0.7271342860456977
test_score 0.6891456533150286


In [425]:
pred = model.predict(test)

In [427]:
submission = {
    'SK_ID_CURR': test.SK_ID_CURR.values,
    'TARGET': pred}

In [428]:
solution = pd.DataFrame(submission)

In [429]:
solution.to_csv('submission.csv',index=False)

### AUC-ROC = 0.59522

In [29]:
feature_str, feature_int = als.get_lists_type_feature(train, list(train))

In [30]:
X = train.drop(columns=['TARGET'])
y = train['TARGET']

In [31]:
TE_encoder = TargetEncoder(cols=feature_str)

In [32]:
X = TE_encoder.fit_transform(X, y)

In [33]:
test = TE_encoder.transform(test)

In [471]:
for train_indices, test_indices in kf.split(X, y):
    X_train, X_val = X.iloc[train_indices], X.iloc[test_indices]
    y_train, y_val = y.iloc[train_indices], y.iloc[test_indices]
    lgbmc = lgb.LGBMClassifier(mobjective='binary',
                               max_depth=30, learning_rate=0.05,
                               n_estimators=100, class_weight='balanced',
                               subsample=0.8,
                               colsample_bytree=0.8,
                               random_state=42,
                               importance_type='gain')
    model = lgbmc.fit(X_train, y_train)
    y_pred_train = model.predict(X_train)
    score_train = roc_auc_score(y_train, y_pred_train)
    print(f'train_score {score_train}')   
    y_pred_val = model.predict(X_val)
    score_val = roc_auc_score(y_val, y_pred_val)
    print(f'test_score {score_val}')

train_score 0.7247083971051533
test_score 0.6866188787108033
train_score 0.725371610403767
test_score 0.6880657965325511
train_score 0.726478741524322
test_score 0.6892735293956554
train_score 0.7256616955419893
test_score 0.6850047696438666
train_score 0.7267795913244837
test_score 0.685272147298716


In [472]:
pred = model.predict(test)

In [473]:
submission = {
    'SK_ID_CURR': test.SK_ID_CURR.values,
    'TARGET': pred}

In [474]:
solution = pd.DataFrame(submission)

In [475]:
solution.to_csv('submission_te.csv',index=False)

### AUC-ROC = 0.65824

In [1]:
#╰( ͡° ͜ʖ ͡° )つ──☆*

In [34]:
train = X

In [35]:
train['Target'] = y

In [36]:
train.shape

(146794, 103)

In [37]:
test.shape

(48744, 102)

In [41]:
test.to_csv('test_base.csv', sep=';', index=False)

In [40]:
train.to_csv('train_base.csv', sep=';', index=False)